## COVID JATIM

- Request

In [1]:
import requests
import pandas as pd
url = "https://radarcovid19.jatimprov.go.id/"
req = requests.get(url)
html_string = req.text

html_string

'<DOCTYPE html>\n<html lang="en">\n<head><script type="text/javascript">CIjs=function(){"use strict";var j,a=function(a,b){return a.className.match(new RegExp("\\b"+b+"\\b"))},b=function(a,b){a.className=a.className.replace(new RegExp("\\b"+b+"\\b")," ")},c=function(b,c){a(b,c)||(b.className+=" "+c)},d=[],e=4,f=0,g=function(){var a=document.querySelector(".ci-toolbar-ajax-requests");if(a){a.textContent="("+d.length+")",a.className="ci-toolbar-ajax-requests";var g=document.querySelector(".ci-toolbar-ajax-info");g&&(g.textContent=d.length+" AJAX request"+(d.length>1?"s":""));var h=document.querySelector(".ci-toolbar-block-ajax");d.length?c(h.parentNode,"active"):b(h.parentNode,"active"),f>0?c(h,"ci-ajax-request-loading"):e<4?(c(h,"ci-toolbar-status-red"),b(h,"ci-ajax-request-loading")):(b(h,"ci-ajax-request-loading"),b(h,"ci-toolbar-status-red"))}},h=function(a){var b=d[a];f++;var c=document.createElement("tr");b.DOMNode=c;var e=document.querySelector(".ci-toolbar-ajax-request-list");if(

- Data Rumah Sakit

In [2]:
rs = html_string.split("var datarumahsakit=")
rs = rs[1].split(";")

rs_df = pd.read_json(rs[0])

rs_df

,terakhir_update,kontak,instansi_rs,iso_vent,iso_tanpa_vent,iso_tanpa_negatif,lat,lon,kab_kota,ketersediaan_vrs,ketersediaan_tvrs,ketersediaan_brs
0,None,,-,0,0,0,0,0,,0,0,0
1,None,0315318000,RS Adi Husada Undaan Wetan,1,2,23,-7.2427746,112.7511014,KOTA SURABAYA,1,2,23
2,05-02-2020 20:42:27,082139840890,RS Bhayangkara H.S. Samsoeri Mertojoso,2,3,0,-7.3284939,112.7261369,KOTA SURABAYA,2,3,0
3,05-02-2020 10:53:55,082139840980,RS Bina Sehat Jember,0,0,13,-8.180509,113.6827395,KAB. JEMBER,0,0,0
4,04-30-2020 18:25:55,082139840923,RS Citra Husada Jember,0,0,8,-8.166839,113.6784772,KAB. JEMBER,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
81,04-21-2020 07:59:00,082139840977,RSUD Syarifah Ambami Rato Ebu Bangkalan,1,10,0,-7.0290495,112.7573814,KAB. BANGKALAN,1,10,0
82,None,(0335 ) 511118,RSUD Tongas,4,53,0,"-7,7339404","113,1126971",KAB. PROBOLINGGO,4,53,0
83,04-28-2020 09:15:14,0341 879879,Rumah Sakit Islam Gondanglegi,0,0,4,-8.1703525,112.6380851,,0,0,2
84,05-01-2020 16:27:57,0315677562,Rumah Sakit Katolik St. Vincentius a Paulo,0,0,1,-7.2907709,112.7338033,KOTA SURABAYA,0,0,0


- Data Observasi

In [3]:
do = html_string.split("var dataobservasi=")
do = do[1].split("];")[0]
do += "]"

do_df = pd.read_json(do)
do_df

,id,kabko,kecamatan,desa_kel,nama_tempat,alamat,daya_tampung,koordinator,nohp,dokumen,keterangan,lat,lon,created_at,updated_at,deleted_at
0,1,KAB. PACITAN,PACITAN,Pacitan,Wisma Atlit,Jl. WR. WR. Supratman No. 16 Lingk. Kriyan Ke...,48 Tempat tidur,Dr. Wahyu Basuki,08992114997,https://covid19dev.jatimprov.go.id/public/uplo...,,-8.1813988,111.0699922,0000-00-00 00:00:00,2020-04-11 19:37:15,0000-00-00 00:00:00
1,2,KAB. PACITAN,PACITAN,Mentoro,Gedung Olahraga,Jl. Magribi No. 37 Desa Mentoro,30 Tempat tidur,Dr. Mashoeri,087736494780,,,"-8,1971148",111.0959702,0000-00-00 00:00:00,2020-04-11 18:28:35,0000-00-00 00:00:00
2,3,KAB. PACITAN,PACITAN,Klepu,Gedung Olahraga,Dsn. Craken Wetan Desa Sumber Sasono Harjo,30 Tempat tidur,Tri Pujianto,082216403762,,,-8.1787029,111.3951706,0000-00-00 00:00:00,2020-04-11 18:43:00,0000-00-00 00:00:00
3,4,KAB. PACITAN,PACITAN,Baleharjo,Ruang Rapat Terminal,Jl. Gatot Subroto,20 Tempat tidur,Wiedi Poornomo,082131308663,,,-8.2119163,111.0993367,0000-00-00 00:00:00,2020-04-11 17:43:31,0000-00-00 00:00:00
4,5,KAB. PACITAN,PACITAN,Purworejo,Ruang PKK Desa,Dsn. Krajan Desa Purworejo,10 Tempat tidur,Samsudin,083122067277,,,-8.1946859,111.1316465,0000-00-00 00:00:00,2020-04-12 07:06:22,0000-00-00 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1292,1297,KAB. PACITAN,DONOROJO,Sendang,,,,,,,,-8.2087115,110.9546794,2020-04-11 17:09:40,2020-04-11 17:09:40,0000-00-00 00:00:00
1293,1298,KAB. TULUNGAGUNG,BOYOLANGU,Beji,,,,,,,,-8.0807472,111.9023209,2020-04-11 19:23:53,2020-04-11 19:23:53,0000-00-00 00:00:00
1294,1299,KAB. TULUNGAGUNG,TULUNGAGUNG,Bago,,,,,,,,-8.0705831,111.9116212,2020-04-11 19:29:15,2020-04-11 19:29:15,0000-00-00 00:00:00
1295,1300,KAB. TULUNGAGUNG,TULUNGAGUNG,Bago,,,,,,,,-8.0705831,111.9116212,2020-04-11 19:29:15,2020-04-11 19:29:15,0000-00-00 00:00:00


- Positive

In [4]:
positive = html_string.split("var datapositiflatlon=")
positive = positive[1].split(";")[0]

positive_df = pd.read_json(positive)

positive_df

,id,status,umur,nomorkasus,kelamin,kabkota,kecamatan,desa,lat,lon,status_pasien
0,627,2,67,Kasus 627,P,,GEDANGAN,,-7.405363,112.725018,MENINGGAL
1,668,0,32,Kasus 663,L,KAB. BANGKALAN,AROSBAYA,LAJING,-6.963909,112.815026,DALAM PERAWATAN
2,880,0,38,Kasus 872,P,KAB. BANGKALAN,BANGIL,RACI,-7.603049,112.824063,DALAM PERAWATAN
3,963,0,30,Kasus 952,P,KAB. BANGKALAN,BANGKALAN,BANCARAN,-7.011922,112.762212,DALAM PERAWATAN
4,339,0,56,Kasus 339,L,KAB. BANGKALAN,BANGKALAN,MLAJAH,"-7,0485540","112,7308820",DALAM PERAWATAN
...,...,...,...,...,...,...,...,...,...,...,...
1034,19,1,25,Kasus 19,P,KOTA SURABAYA,WONOKROMO,SAWUNGGALING,"-7,2994400","112,7251800",SEMBUH
1035,210,1,66,Kasus 210,P,KOTA SURABAYA,WONOKROMO,SAWUNGGALING,"-7,2982334","112,7227870",SEMBUH
1036,1038,0,50,Kasus 1026,P,KOTA SURABAYA,WONOKROMO,WONOKROMO,-7.301797,112.729548,DALAM PERAWATAN
1037,537,0,45,Kasus 537,P,KOTA SURABAYA,WONOKROMO,WONOKROMO,"-7,3009840","112,7320940",DALAM PERAWATAN


In [5]:
#Minify
rss = rs_df.copy()
rss = rss.rename(columns={
    'u': 'terakhir_update', 
    'instansi_rs': 'n', 
    'kontak': 'k',
    'iso_tanpa_vent': 'iso',
    'iso_vent': 'isov',
    'iso_tanpa_negatif': 'isotn',
    'lat': 'lat',
    'lon': 'lng',
    'kab_kota': 'kab',
    'ketersediaan_vrs': 'vrs',
    'ketersediaan_tvrs': 'tvrs',
    'ketersediaan_brs': 'brs'
})

rss

,terakhir_update,k,n,isov,iso,isotn,lat,lng,kab,vrs,tvrs,brs
0,None,,-,0,0,0,0,0,,0,0,0
1,None,0315318000,RS Adi Husada Undaan Wetan,1,2,23,-7.2427746,112.7511014,KOTA SURABAYA,1,2,23
2,05-02-2020 20:42:27,082139840890,RS Bhayangkara H.S. Samsoeri Mertojoso,2,3,0,-7.3284939,112.7261369,KOTA SURABAYA,2,3,0
3,05-02-2020 10:53:55,082139840980,RS Bina Sehat Jember,0,0,13,-8.180509,113.6827395,KAB. JEMBER,0,0,0
4,04-30-2020 18:25:55,082139840923,RS Citra Husada Jember,0,0,8,-8.166839,113.6784772,KAB. JEMBER,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
81,04-21-2020 07:59:00,082139840977,RSUD Syarifah Ambami Rato Ebu Bangkalan,1,10,0,-7.0290495,112.7573814,KAB. BANGKALAN,1,10,0
82,None,(0335 ) 511118,RSUD Tongas,4,53,0,"-7,7339404","113,1126971",KAB. PROBOLINGGO,4,53,0
83,04-28-2020 09:15:14,0341 879879,Rumah Sakit Islam Gondanglegi,0,0,4,-8.1703525,112.6380851,,0,0,2
84,05-01-2020 16:27:57,0315677562,Rumah Sakit Katolik St. Vincentius a Paulo,0,0,1,-7.2907709,112.7338033,KOTA SURABAYA,0,0,0


In [6]:
dos = do_df.copy()
dos = dos.drop(["id", "created_at", "updated_at", "deleted_at"], axis=1)
dos = dos.rename(columns={
    'kabko': 'kab',
    'kecamatan': 'kec',
    'desa_kel': 'kel',
    'nama_tempat': 'n',
    'alamat': 'a',
    'daya_tampung': 'dt',
    'koordinator': 'koor',
    'nohp': 'hp',
    'dokumen': 'dok',
    'keterangan': 'ket',
    'lat': 'lat',
    'lon': 'lng'
})

dos

,kab,kec,kel,n,a,dt,koor,hp,dok,ket,lat,lng
0,KAB. PACITAN,PACITAN,Pacitan,Wisma Atlit,Jl. WR. WR. Supratman No. 16 Lingk. Kriyan Ke...,48 Tempat tidur,Dr. Wahyu Basuki,08992114997,https://covid19dev.jatimprov.go.id/public/uplo...,,-8.1813988,111.0699922
1,KAB. PACITAN,PACITAN,Mentoro,Gedung Olahraga,Jl. Magribi No. 37 Desa Mentoro,30 Tempat tidur,Dr. Mashoeri,087736494780,,,"-8,1971148",111.0959702
2,KAB. PACITAN,PACITAN,Klepu,Gedung Olahraga,Dsn. Craken Wetan Desa Sumber Sasono Harjo,30 Tempat tidur,Tri Pujianto,082216403762,,,-8.1787029,111.3951706
3,KAB. PACITAN,PACITAN,Baleharjo,Ruang Rapat Terminal,Jl. Gatot Subroto,20 Tempat tidur,Wiedi Poornomo,082131308663,,,-8.2119163,111.0993367
4,KAB. PACITAN,PACITAN,Purworejo,Ruang PKK Desa,Dsn. Krajan Desa Purworejo,10 Tempat tidur,Samsudin,083122067277,,,-8.1946859,111.1316465
...,...,...,...,...,...,...,...,...,...,...,...,...
1292,KAB. PACITAN,DONOROJO,Sendang,,,,,,,,-8.2087115,110.9546794
1293,KAB. TULUNGAGUNG,BOYOLANGU,Beji,,,,,,,,-8.0807472,111.9023209
1294,KAB. TULUNGAGUNG,TULUNGAGUNG,Bago,,,,,,,,-8.0705831,111.9116212
1295,KAB. TULUNGAGUNG,TULUNGAGUNG,Bago,,,,,,,,-8.0705831,111.9116212


In [7]:

positives = positive_df.copy()
positives = positives.drop("status", axis=1)
positives = positives.rename(columns={
    'umur': 'u',
    'kelamin': 'k',
    'kabkota': 'kab',
    'kecamatan': 'kec',
    'desa': 'kel',
    'lat': 'lat',
    'lon': 'lng',
    'status_pasien': 'sp'
})

positives


,id,u,nomorkasus,k,kab,kec,kel,lat,lng,sp
0,627,67,Kasus 627,P,,GEDANGAN,,-7.405363,112.725018,MENINGGAL
1,668,32,Kasus 663,L,KAB. BANGKALAN,AROSBAYA,LAJING,-6.963909,112.815026,DALAM PERAWATAN
2,880,38,Kasus 872,P,KAB. BANGKALAN,BANGIL,RACI,-7.603049,112.824063,DALAM PERAWATAN
3,963,30,Kasus 952,P,KAB. BANGKALAN,BANGKALAN,BANCARAN,-7.011922,112.762212,DALAM PERAWATAN
4,339,56,Kasus 339,L,KAB. BANGKALAN,BANGKALAN,MLAJAH,"-7,0485540","112,7308820",DALAM PERAWATAN
...,...,...,...,...,...,...,...,...,...,...
1034,19,25,Kasus 19,P,KOTA SURABAYA,WONOKROMO,SAWUNGGALING,"-7,2994400","112,7251800",SEMBUH
1035,210,66,Kasus 210,P,KOTA SURABAYA,WONOKROMO,SAWUNGGALING,"-7,2982334","112,7227870",SEMBUH
1036,1038,50,Kasus 1026,P,KOTA SURABAYA,WONOKROMO,WONOKROMO,-7.301797,112.729548,DALAM PERAWATAN
1037,537,45,Kasus 537,P,KOTA SURABAYA,WONOKROMO,WONOKROMO,"-7,3009840","112,7320940",DALAM PERAWATAN


In [8]:
#Rename latest json to yesterday suffix
#export new json to latest suffix
import os
import datetime

now = datetime.datetime.now()
now_suffix = now.strftime('%Y-%m-%d')
yesterday = now - datetime.timedelta(days = 1)
yesterday_suffix = yesterday.strftime('%Y-%m-%d')

try:
    os.rename('rs_latest.json', 'rs_' + yesterday_suffix + '.json')
    os.rename('do_latest.json', 'do_' + yesterday_suffix + '.json')
    os.rename('pos_latest.json', 'pos_' + yesterday_suffix + '.json')
except:
    pass


new_rss = rss.to_json(orient="records")
new_rss = '{"date": "' + now_suffix + '", "data":' + new_rss + '}'
rss_file = open("rs_latest.json", "w")
rss_file.write(new_rss)
rss_file.close()


new_dos = dos.to_json(orient="records")
new_dos = '{"date": "' + now_suffix + '", "data":' + new_dos + '}'
dos_file = open("do_latest.json", "w")
dos_file.write(new_dos)
dos_file.close()

new_pos = positives.to_json(orient="records")
new_pos = '{"date": "' + now_suffix + '", "data":' + new_pos + '}'
pos_file = open("pos_latest.json", "w")
pos_file.write(new_pos)
pos_file.close()